In [11]:
import os
import cv2
import numpy as np
from imutils import paths

In [12]:
from sklearn.preprocessing import LabelBinarizer

In [30]:
from sklearn.model_selection import train_test_split

In [33]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type

In [47]:
datapath = r"D:\VSCODE\Git Upload\Sports-Video-Classification\data"
outputmodel = r"D:\VSCODE\Git Upload\Sports-Video-Classification\model\videoClassificationModel"
outputlabinarizer = r"D:\VSCODE\Git Upload\Sports-Video-Classification\videoClassificationBinarizer"

In [28]:
Sports_Labels = set(["boxing", "swimming", "table_tennis"])
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    data.append(image)
    labels.append(label)

In [29]:
data = np.array(data)
labels = np.array(labels)

lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [32]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size = 0.3, stratify = labels, random_state = 42)

In [34]:
trainingAugmentation = ImageDataGenerator(
    rotation_range = 30,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = "nearest"
)

validationAugmentation = ImageDataGenerator()

mean = np.array([123.68, 116.779, 103.939], dtype = "float32")

trainingAugmentation.mean = mean
validationAugmentation.mean = mean

In [35]:
from keras.applications import ResNet50

from keras.layers import Input
from keras.layers.pooling import AveragePooling2D

from keras.layers.core import Flatten
from keras.layers.core import Dense

from keras.layers.core import Dropout

from keras.models import Model 

In [36]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)

model = Model(inputs = baseModel.input, outputs= headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers.trainable = False

E:\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 226s 2us/step



In [37]:
from keras.optimizers import SGD

In [39]:
epoch = 25

In [40]:
opt = SGD(lr=0.0001, momentum=0.9, decay=1e-4/epoch)

In [41]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [42]:
History = model.fit_generator(
    trainingAugmentation.flow(X_train, Y_train, batch_size = 32),
    steps_per_epoch = len(X_train) // 32,
    validation_data = validationAugmentation.flow(X_test, Y_test),
    validation_steps = len(X_test) // 32,
    epochs = epoch
)


Epoch 1/25
45/45 [==============================] - 116s 3s/step - loss: 1.1326 - accuracy: 0.4637 - val_loss: 1.0054 - val_accuracy: 0.5691
Epoch 2/25
45/45 [==============================] - 68s 2s/step - loss: 0.8408 - accuracy: 0.6250 - val_loss: 0.8276 - val_accuracy: 0.6913
Epoch 3/25
45/45 [==============================] - 66s 1s/step - loss: 0.7026 - accuracy: 0.7018 - val_loss: 0.6759 - val_accuracy: 0.7768
Epoch 4/25
45/45 [==============================] - 66s 1s/step - loss: 0.5888 - accuracy: 0.7689 - val_loss: 0.3472 - val_accuracy: 0.8255
Epoch 5/25
45/45 [==============================] - 66s 1s/step - loss: 0.4897 - accuracy: 0.8115 - val_loss: 0.4554 - val_accuracy: 0.8406
Epoch 6/25
45/45 [==============================] - 66s 1s/step - loss: 0.4880 - accuracy: 0.8101 - val_loss: 0.3112 - val_accuracy: 0.8574
Epoch 7/25
45/45 [==============================] - 66s 1s/step - loss: 0.4319 - accuracy: 0.8324 - val_loss: 0.2736 - val_accuracy: 0.8876
Epoch 8/25
45/45 [

In [43]:
import pickle

In [48]:
model.save(outputmodel)
lbinarizer = open(r"D:\VSCODE\Git Upload\Sports-Video-Classification\model\videoclassificationbinarizer.pickle", "wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()